# SimiC Preprocessing Pipeline Tutorial

>*Author: Irene Marín-Goñi, PhD student - ML4BM group (CIMA University of Navarra)*

This notebook demonstrates how to preprocess single-cell RNA-seq data for SimiC analysis.

## Overview
This preprocessing tutorial covers:
1. Package installation and setup
2. MAGIC imputation pipeline
3. Gene selection and experiment setup
4. Preparing input files for SimiC

For running SimiC analysis see `Tutorial_SimiCPipeline_simple.ipynb`

## Introduction
Before running SimiC, you need to:
- Impute your scRNA-seq data. We recommend to use [MAGIC](https://pypi.org/project/magic-impute/3.0.0/) and include a wrapper class `MagicPipeline` to ease the process.
- Select top variable genes based on MAD (Median Absolute Deviation).
- Prepare input files in the correct format.

This tutorial shows you how to do all of this using the SimiC preprocessing modules.

## Setup

<div class="alert alert-block alert-warning">
<b>Warning: Need to include here Installing instructions (github/Docker/dependencies)</b>
</div>

Required packages:
- anndata
- pandas
- numpy
- scipy
- scprep
- magic-impute

## Import Modules
First, import the necessary preprocessing modules.

In [ ]:
import sys
import anndata as ad
import pandas as pd
from pathlib import Path

# Add source directory to path
sys.path.append('./src/simicpipeline/core/')

from simicpreprocess import MagicPipeline, ExperimentSetup

# Part 1: MAGIC Imputation Pipeline

MAGIC (Markov Affinity-based Graph Imputation of Cells) is used to denoise and impute scRNA-seq data. `MagicPipeline` facilitates the steps described in [Magic Tuotrial]("https://nbviewer.org/github/KrishnaswamyLab/magic/blob/master/python/tutorial_notebooks/emt_tutorial.ipynb")

## Step 1.1: Load Your Data

Load your raw expression data. The data can be in AnnData format (cells × genes) or pandas DataFrame. 

<div class="alert alert-block alert-warning">
<b>If you have processsed your data alreadey, make sure the adata object has the raw counts in the `adata.X` slot</b>
</div>


In [ ]:
# Example: Load from h5ad file
# adata = ad.read_h5ad('path/to/your/data.h5ad')

# Example: Load from 10X format
# adata = ad.read_10x_mtx('path/to/10x/directory')

# Example: Load from CSV files
# expression_data = pd.read_csv('path/to/expression_data.csv', index_col=0)
# metadata = pd.read_csv('path/to/metadata.csv', index_col=0)
# adata = ad.AnnData(X=expression_data.values, obs=metadata)

# Example: Load from Matrix Market format

# df = simicpipeline.io.load_from_matrix_market( 
#     matrix_path=Path("path/to/matrix.mtx"),
#     genes_path=Path("path/to/genes.txt"),
#     cells_path=Path("path/to/cells.txt"),
#     transpose=False,
#     cells_index_name="Cell",
# )
# adata = ad.AnnData(X=df.values, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))

print("Load your AnnData object here")
# adata = ...

## Step 1.2: Initialize MAGIC Pipeline

Create a MAGIC pipeline instance:
- `input_data`: Your AnnData object or pandas DataFrame (cells × genes). Raw counts should be in `adata.X`
- `project_dir`: Project directory where `magic_output`dir will be created and files will be saved
- `magic_output_file`: Filename for the imputed data (default: 'magic_data_allcells_sqrt.pickle')
- `filtered`: Set to True if data is already filtered (default: False)

In [ ]:
# This command will initialize the MAGIC pipeline and generate the output directory if it does not exist
magic_pipeline = MagicPipeline(
    input_data= adata,
    project_dir='./SimiCExampleRun',
    magic_output_file='magic_imputed.pickle',
    filtered=False
)

print(magic_pipeline)

## Step 1.3: Filter Cells and Genes

Remove low-quality cells and lowly-expressed genes:
- `min_cells_per_gene`: Minimum number of cells expressing a gene (default: 10)
- `min_umis_per_cell`: Minimum total UMI counts per cell (default: 500)

<em>If your data was already filtered you can skip this step and set the flitered flag to `True` in the previous step.</em>

In [ ]:
magic_pipeline.filter_cells_and_genes( min_cells_per_gene = 10, min_umis_per_cell = 500)

## Step 1.4: Normalize Data

Perform library size normalization with `scprep` followed by square root transformation.

In [ ]:
magic_pipeline.normalize_data()

## Step 1.5: Run MAGIC Imputation

Run MAGIC imputation with custom parameters:
- `t`: Number of diffusion steps (default: 'auto')
- `knn`: Number of nearest neighbors (default: 5)
- `decay`: Decay rate for kernel (default: 1)
- `n_jobs`: Number of parallel jobs (default: -2)
- `genes`: Genes to be returned. If None or "all genes" it returns teh entire matrix.
- `save_data`: Whether to automatically save imputed data (default: True). If magic_output_file extension is .pickle will save it in .pickle, if h5ad, will save in adata format.

See [MAGIC documentation](https://magic.readthedocs.io/) for more parameter options.

In [ ]:
magic_pipeline.run_magic(
    random_state=123,
    n_jobs=-2,  # Use all but 1 CPU cores
    save_data=True
)

## Step 1.6: Check Pipeline Status

In [ ]:
magic_pipeline.get_pipeline_status()
# print(f"Data loaded: {status['data_loaded']}")
# print(f"Filtered: {status['filtered']}")
# print(f"Imputed: {status['imputed']}")
# print(f"Cells: {status['n_cells']}")
# print(f"Genes: {status['n_genes']}")
# print(f"MAGIC data available: {status['magic_data_available']}")

<div class="alert alert-block alert-success">
<b>Success!</b> MAGIC imputation is complete. The imputed data is saved in the magic_output directory.
</div>



# Part 2: Experiment Setup and Gene Selection

Now we'll select top variable genes and prepare input files for SimiC. 

<em>Note that if you have your data filtered, normalized and inputed with other methods you can start from here.</em>

## Step 2.1: Load MAGIC-Imputed Data

Load the imputed data from the MAGIC pipeline.

In [ ]:
# Access the MAGIC-imputed AnnData object
imputed_data = magic_pipeline.magic_adata

# Or load from saved file
# import pickle
# with open('./SimiCExampleRun/magic_output/magic_imputed.pickle', 'rb') as f:
#     imputed_data = pickle.load(f)

print(f"Imputed data shape: {imputed_data.shape}")

## Step 2.2: Initialize Experiment Setup

Create an experiment setup instance:
- `input_data`: Your imputed AnnData object or pandas DataFrame (cells × genes)
- `tf_path`: Path to transcription factor list file (.csv or .txt)
- `project_dir`: Directory where experiment files will be saved

This will automatically create the SimiC directory structure:
```
project_dir/
├── inputFiles/       # Input files for SimiC
└── outputSimic/
    ├── figures/      # For future visualizations
    └── matrices/     # For future results
```

In [ ]:
experiment = ExperimentSetup(
    input_data=imputed_data,
    tf_path='./data/transcription_factors.csv',
    project_dir='./SimiCExampleRun'
)

print(f"Matrix shape: {experiment.matrix.shape}")
print(f"Number of cells: {len(experiment.cell_names)}")
print(f"Number of genes: {len(experiment.gene_names)}")
print(f"Number of TFs: {len(experiment.tf_list)}")

## Step 2.3: Calculate MAD and Select Genes

Select top variable genes based on Median Absolute Deviation (MAD):
- `n_tfs`: Number of top TF genes to select (default: 100)
- `n_targets`: Number of top target genes to select (default: 1000)

Returns tuple of (TF_list, TARGET_list)

In [ ]:
tf_list, target_list = experiment.calculate_mad_genes(
    n_tfs=100,
    n_targets=1000
)

print(f"Selected {len(tf_list)} TFs")
print(f"Selected {len(target_list)} targets")
print(f"\nTop 10 TFs: {tf_list[:10]}")
print(f"\nTop 10 targets: {target_list[:10]}")

## Step 2.4: Subset Data to Selected Genes

Create a subset of your data containing only the selected TFs and targets.

In [ ]:
# Combine TF and target lists
selected_genes = tf_list + target_list

# Subset the data
if isinstance(imputed_data, ad.AnnData):
    subset_data = imputed_data[:, selected_genes].copy()
elif isinstance(imputed_data, pd.DataFrame):
    subset_data = imputed_data[selected_genes].copy()

print(f"Subset data shape: {subset_data.shape}")

## Step 2.5: Save Experiment Files

Save the expression matrix and TF names as `.pickle` files and annotation (optional) as `.txt` file
- `run_data`: `ad.AnnData` or `pd.Dataframe` with data to run in SimiC (Inputed and sliced according to experiment run)
- `matrix_filename`: Filename to save `run_data` (saved with row/column headers)
- `tf_filename`: Filename for TF names list
- `annotation`: if `run_data` is `ad.AnnData` and `anotation` is in `run_dataobs.columns` it will create a `.txt` file with the phenotype annotations needed for SimiC.

All files are saved in the `inputFiles/` directory.

In [ ]:
experiment.save_experiment_files(
    run_data=subset_data,
    matrix_filename='expression_matrix.pickle',
    tf_filename='TF_list.pickle',
    annotation='groups'
)

<div class="alert alert-block alert-success">
<b>Success!</b> All preprocessing is complete. Your files are ready for SimiC analysis.
</div>

## Step 2.7: Verify Saved Files

Check that all files were created correctly.

In [ ]:
import os

input_dir = experiment.input_files_dir
print(f"Input files directory: {input_dir}")
print(f"\nFiles created:")
for file in os.listdir(input_dir):
    filepath = input_dir / file
    size = os.path.getsize(filepath) / 1024  # Size in KB
    print(f"  - {file} ({size:.2f} KB)")

# Preview the expression matrix
expr_matrix = pd.read_csv(input_dir / 'expression_matrix.csv', index_col=0)
print(f"\nExpression matrix preview (first 5 rows and columns):")
print(expr_matrix.iloc[:5, :5])

## Summary

This tutorial covered:
✓ Loading and filtering scRNA-seq data
✓ Running MAGIC imputation
✓ Selecting top variable genes using MAD
✓ Preparing input files for SimiC analysis
✓ Creating the proper directory structure

### Output Directory Structure

Your output directory now contains:
```
SimicExampleRun/
├── magic_output/
│   └── magic_imputed.pickle
├── inputFiles/
│   ├── expression_matrix.pickle
│   ├── TF_list.pickle
│   └── groups_phenotype.txt
└── outputSimic/
    ├── figures/
    └── matrices/
```
## Alternative Steps

In this tutorial we used the whole Magic-inputed matrix (## Part 1) and selected top MAD genes but you may want to run SimiC in a subset of cells from your data. Make sure you slice the data before you inilitalized the `ExperimentSetup` class so MAD genes are calculated over your cells of interest.

All these steps are optional but recommended before running SimiCPipeline just:
1. Make sure to **prepare cell assignments** if not done before: Create a file with cell phenotype labels matching your expression matrix.
2. Prepare **expression matrix** pickle file
3. Prepare **TF_list** pickle with TF genes in your expression matrix.

## Next steps:
1. **Run SimiC!**: Use `SimiCPipeline` class to run SimiC.
2. **Explore results**: Use `SimicVisualization`class to analyze GRNs and TF activities.

Check `Tutorial_SimiCPipeline_simple.ipynb` or `Tutorial_SimiCPipeline_full` for more info.


### Important Notes

<div class="alert alert-block alert-info">
<b>Data Format:</b> All matrices are stored as cells × genes (rows = cells, columns = genes)
</div>

<div class="alert alert-block alert-warning">
<b>Memory Usage:</b> MAGIC imputation can be memory-intensive for large datasets. Consider:
- Using a machine with sufficient RAM
- Adjusting MAGIC parameters (n_jobs, knn, t)
</div>